In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [9]:
import psycopg2
import pandas as pd

In [187]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=text123")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create sparfly database with UTFB encoding
    cur.execute('DROP DATABASE IF EXISTS worldcup')
    cur.execute('CREATE DATABASE worldcup')
    
    #close connection
    conn.close()
    
    #connect to sparkfly database
    conn = psycopg2.connect("host=localhost dbname=worldcup user=postgres password=text123")
    cur = conn.cursor()
    
    return cur, conn
    

In [188]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [189]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [69]:
Attendance = pd.read_csv('Attendance.csv')

In [70]:
Attendance.head()

,Ranking,Year,Hosts,Total_Attendance,Matches,Average_Attendance,Number,Venue,Game(s)
0,0,1930,Uruguay,590549,18,32808,93000,"Estadio Centenario, Montevideo","Uruguay 6–1 Yugoslavia, Semi-final"
1,1,1934,Italy,363000,17,21353,55000,"Stadio Nazionale PNF, Rome","Italy 2–1 Czechoslovakia, Final"
2,2,1938,France,375700,18,20872,58455,"Olympique de Colombes, Paris","France 1–3 Italy, Quarter-final"
3,3,1950,Brazil,1045246,22,47511,"173,850[86]","Maracanã Stadium, Rio de Janeiro","Brazil 1–2 Uruguay, Deciding match"
4,4,1954,Switzerland,768607,26,29562,63000,"Wankdorf Stadium, Bern","West Germany 3–2 Hungary, Final"


In [128]:
Awards = pd.read_csv('Award.csv')

In [129]:
Awards.head()

,Ranking,World _Cup,Golden _Ball,Golden _Boot,Goals,Golden _Glove,Clean _sheets,FIFA _Young _Player_ Award,FIFA _Fair _Play _Trophy
0,0,1930 Uruguay,Not Awarded,Guillermo Stábile,8,Not Awarded,0,Not Awarded,Not Awarded
1,1,1934 Italy,Not Awarded,Oldřich Nejedlý,5,Not Awarded,0,Not Awarded,Not Awarded
2,2,1938 France,Not Awarded,Leônidas,7,Not Awarded,0,Not Awarded,Not Awarded
3,3,1950 Brazil,Not Awarded,Ademir,8,Not Awarded,0,Not Awarded,Not Awarded
4,4,1954 Switzerland,Not Awarded,Sándor Kocsis,11,Not Awarded,0,Not Awarded,Not Awarded


In [130]:
finals = pd.read_csv('finals.csv')

In [131]:
finals.head()

,Ranking,Year,Host,Champion,Score,Runner_up,Third,Score.1,Fourth,No. _ofteams
0,0,1930,Uruguay,Uruguay,"4–2 Estadio Centenario, Montevideo",Argentina,United States,– [n 1],Yugoslavia,13
1,1,1934,Italy,Italy,"2–1 (a.e.t.) Stadio Nazionale PNF, Rome",Czechoslovakia,Germany,"3–2 Stadio Giorgio Ascarelli, Naples",Austria,16
2,2,1938,France,Italy,"4–2 Stade de Colombes, Paris",Hungary,Brazil,"4–2 Parc Lescure, Bordeaux",Sweden,15
3,3,1942,(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II)
4,4,1946,(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II),(Not held because of World War II)


In [190]:
cur, conn = create_database()

In [191]:
attendance_data_table_create = ("""CREATE TABLE IF NOT EXISTS attendance(
                               Ranking VARCHAR PRIMARY KEY,
                               Year VARCHAR,
                               Host VARCHAR,
                               Total_Attendance VARCHAR,
                               Matches VARCHAR,
                               Average_Attendance VARCHAR,
                               Number VARCHAR, 
                               Venue VARCHAR,
                               Games VARCHAR
                               )""")

In [192]:
cur.execute(attendance_data_table_create)
conn.commit()

In [193]:
awards_data_table_create = ("""CREATE TABLE IF NOT EXISTS awards(
                               Ranking VARCHAR PRIMARY KEY,
                               World_Cup VARCHAR,
                               Goldem_Ball VARCHAR,
                               Goldem_Boot VARCHAR,
                               Goals VARCHAR,
                               Golden_Glove VARCHAR,
                               Cleen_Sheets VARCHAR,
                               FIFA_Young_Player_Award VARCHAR,
                               FIFA_Fair_Play_Trophy VARCHAR
                               )""")


In [194]:
cur.execute(awards_data_table_create)
conn.commit()

In [195]:
finals_data_table_create = ("""CREATE TABLE IF NOT EXISTS finals(
                               Ranking VARCHAR PRIMARY KEY,
                               Year VARCHAR,
                               Host VARCHAR,
                               Champion VARCHAR,
                               Score VARCHAR,
                               Runner_up VARCHAR,
                               Third VARCHAR,
                               Score1 VARCHAR,
                               Fourth VARCHAR,
                               No_ofteams VARCHAR
                               )""")
cur.execute(finals_data_table_create)
conn.commit()

In [196]:
attendance_data_table_insert = ("""INSERT INTO attendance(
                               Ranking ,
                               Year ,
                               Host ,
                               Total_Attendance ,
                               Matches ,
                               Average_Attendance,
                               Number , 
                               Venue ,
                               Games)
                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                               """)

In [197]:
for i, row in Attendance.iterrows():
    cur.execute(attendance_data_table_insert, list(row))

In [198]:
conn.commit()

In [199]:
awards_data_table_insert = ("""INSERT INTO awards(
                               Ranking ,
                               World_Cup ,
                               Goldem_Ball ,
                               Goldem_Boot ,
                               Goals ,
                               Golden_Glove ,
                               Cleen_Sheets ,
                               FIFA_Young_Player_Award ,
                               FIFA_Fair_Play_Trophy)
                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
                               """)
for i, row in Awards.iterrows():
    cur.execute(awards_data_table_insert, list(row))
conn.commit()

In [200]:
finals_data_table_insert = ("""INSERT INTO finals(
                               Ranking ,
                               Year ,
                               Host ,
                               Champion ,
                               Score ,
                               Runner_up ,
                               Third ,
                               Score1 ,
                               Fourth ,
                               No_ofteams)
                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%)
                               """)
for i, row in finals.iterrows():
    cur.execute(finals_data_table_insert, list(row))
conn.commit()

ValueError: unsupported format character ')' (0x29) at index 490